In [35]:
pip install keras-tuner

                                              0.0/129.1 kB ? eta -:--:--
     ---------------------------             92.2/129.1 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 129.1/129.1 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing the required libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import joblib
import os

In [2]:
# Creating our X_train array 
import csv

Euler_KAM = []
for file_index in range(1, 20):
    if file_index == 6:  # Skipping file 6
        continue
    with open(fr"IPF\IPF_Final_{file_index}.csv", "r") as file:
        contents = csv.DictReader(file, delimiter=",")
        Euler_at_fixed_strain = []
        for row in contents:
            Euler_at_fixed_strain.append([
                float(row["Phi1"]), 
                float(row["Phi"]), 
                float(row["Phi2"]), 
            ])
        Euler_KAM.append(Euler_at_fixed_strain)

# Convert to numpy array and reshape appropriately
X_train = np.array(Euler_KAM).transpose((1, 0, 2))  # Shape: (samples, time_steps, features)

# Reshape for RNN input
# As we always have a 3-D array as an input to a LSTM network
# The first dimension represents batch size
# The second dimension represents time steps
# The third dimension represents the number of units in one input sequence


In [3]:
np.shape(X_train)

(10004, 18, 3)

In [4]:
# Creating our Y _train array

Y_train = []
df = pd.read_csv(r"IPF\IPF_Final_20.csv")
num_cells = len(df['Phi1'])

for i in range(num_cells):
    Y_train.append([
        df['Phi1'].iloc[i], 
        df['Phi'].iloc[i], 
        df['Phi2'].iloc[i]
    ])

Y_train = np.array(Y_train)  # Shape: (samples, 3)

In [5]:
np.shape(Y_train)

(10004, 3)

In [6]:
# Splitting our data into training-testing batches
# and adjusting their shapes according to the model requirements
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [7]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

(8003, 18, 3)
(8003, 3)
(2001, 18, 3)
(2001, 3)


In [ ]:
# Initialization and training of our model

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
import requests

# Define input layer
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

# Separate LSTM layers for each Euler angle
lstm_phi1 = LSTM(64, return_sequences=False, name="lstm_phi1")(input_layer)
lstm_phi = LSTM(64, return_sequences=False, name="lstm_phi")(input_layer)
lstm_phi2 = LSTM(64, return_sequences=False, name="lstm_phi2")(input_layer)

# Combine LSTM outputs to capture correlations
combined_representation = Concatenate(name="combined_representation")([lstm_phi1, lstm_phi, lstm_phi2])

# Final Dense layers for each Eule*r angle
phi1_output = Dense(1, name="phi1_output")(combined_representation)
phi_output = Dense(1, name="phi_output")(combined_representation)
phi2_output = Dense(1, name="phi2_output")(combined_representation)

# Define the model
model = Model(inputs=input_layer, outputs=[phi1_output, phi_output, phi2_output])

# Compile model with weighted loss function
model.compile(
    loss={"phi1_output": "mse", "phi_output": "mse", "phi2_output": "mse"},
    loss_weights={"phi1_output": 0.33, "phi_output": 0.33, "phi2_output": 0.34},
    optimizer=Adam(learning_rate=0.001)
)

# Print model summary
model.summary()

# Train the model
history = model.fit(
    x_train, [y_train[:, 0], y_train[:, 1], y_train[:, 2]],
    validation_data=(x_test, [y_test[:, 0], y_test[:, 1], y_test[:, 2]]),
    epochs=100,
    batch_size=32
)

# Save the model
model.save("Tension_woKAM_EnhancedLSTM.h5")
bot_token = "##########:#####################-#############"
chat_id = "##########"
message = "Model has been trained and saved to an h5 file."
url = f"https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={message}"
requests.get(url)


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)    │ (None, 18, 3)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_phi1 (LSTM)              │ (None, 64)                │          17,408 │ input_layer_7[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_phi (LSTM)               │ (None, 64)                │          17,408 │ input_layer_7[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_phi2 (LSTM)              │ (None, 64)                │          17,408 │ input_layer_7[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ combined_representation       │ (None, 192)               │               0 │ lstm_phi1[0][0],           │
│ (Concatenate)                 │                           │                 │ lstm_phi[0][0],            │
│                               │                           │                 │ lstm_phi2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ phi1_output (Dense)           │ (None, 1)                 │             193 │ combined_representation[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ phi_output (Dense)            │ (None, 1)                 │             193 │ combined_representation[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ phi2_output (Dense)           │ (None, 1)                 │             193 │ combined_representation[0… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 52,803 (206.26 KB)

 Trainable params: 52,803 (206.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 8191.4028 - val_loss: 4669.4136
Epoch 2/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 4035.0986 - val_loss: 3170.4270
Epoch 3/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 2794.7288 - val_loss: 2573.6826
Epoch 4/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 2211.6819 - val_loss: 2187.2773
Epoch 5/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1976.0925 - val_loss: 1903.1748
Epoch 6/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1779.2910 - val_loss: 1672.9886
Epoch 7/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1505.3976 - val_loss: 1463.7427
Epoch 8/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1208.4847 - val_loss: 1283.7524
Epoch 9/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1208.5049 - val_loss: 1116.3844
Epoch 10/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 946.4760 - val_loss: 983.6096
Epoch 11/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 735.8547 

In [ ]:
# Defining a function to make the predictions for a particular strain value

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import csv

def predictions(to_predict):
    complete_features = []

    # Load all required data
    for file_index in range(1, 20):
        if file_index == 6:  # Skipping file 6
            continue
        with open(fr"IPF\IPF_Final_{file_index}.csv", "r") as file:
            contents = csv.DictReader(file, delimiter=",")
            features = []
            X_coordinates = []
            y_coordinates = []
            for row in contents:
                features.append([
                    float(row["Phi1"]), 
                    float(row["Phi"]), 
                    float(row["Phi2"]),
                ])
                X_coordinates.append(row["X"])
                y_coordinates.append(row["Y"])
            complete_features.append(features)

    # Convert to numpy array and reshape appropriately
    X_train = np.array(complete_features)
    print("X_train shape before reshaping:", X_train.shape)  # Debugging output

    # Ensure the shape matches (samples, time_steps, features)
    X_train = X_train.transpose((1, 0, 2))
    print("X_train shape after reshaping:", X_train.shape)  # Debugging output
    
    X_copy = X_train.copy()
    outer_len = len(X_copy)
    for file_index in range(20, to_predict):
        df = pd.read_csv(fr"IPF\IPF_pred_cleaned_woKAM_modifiedLSTM\IPF_pred_{file_index}_cleaned.csv")
        latest = df[["Phi1", "Phi", "Phi2"]] 

        X_copy = np.roll(X_copy, shift=-1, axis=1)  # Shift all timesteps left
        X_copy[:, -1, :] = latest.values  # Replace the last time step with new values


    # Load pre-trained model
    model = load_model("Tension_woKAM_EnhancedLSTM.h5", compile=False)

    predicted_values = []

    # Predict for each sample
    for i in range(outer_len):  # Iterating over samples
        sample_input = X_copy[i].reshape(1, X_copy.shape[1], X_copy.shape[2])  # Reshape for LSTM
        print(i)
        pred = np.array([1])
        for j in model.predict(sample_input):
            pred = np.append(pred, j)
        pred = np.append(pred, y_coordinates[i])
        pred = np.append(pred, X_coordinates[i])
#         print(pred)
        predicted_values.append(pred)

    predicted_values = np.array(predicted_values)
    predicted_df = pd.DataFrame(predicted_values, columns=["Phase", "Phi1", "Phi", "Phi2", "Y", "X"])
    
    return predicted_df



In [ ]:
import requests

try:
    to_predict = int(input("Enter the strain value for which the predictions are to be made: "))
    predicted_features = predictions(to_predict)
    bot_token = "##########:#####################-#############"
    chat_id = "##########"
    message = "Predictions have been smoothened and saved."
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={message}"
    requests.get(url)
    
except Exception as e:
    print(e)
    error_message = f"ERROR: {str(e)}"
    error_url = f"https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={error_message}"
    requests.get(error_url)

Enter the strain value for which the predictions are to be made: 21
X_train shape before reshaping: (18, 10004, 3)
X_train shape after reshaping: (10004, 18, 3)
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
108
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
109
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
110
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
111
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
112
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
113
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
114
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
115
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
116
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
118
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
119
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
120
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
121
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
122
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
123
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
124
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
125
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
127
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
128
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
129
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
130
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
215
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
216
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
218
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
219
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
221
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
222
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
223
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
224
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
225
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
226
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
227
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
228
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
229
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
230
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
231
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
232
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
233
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
235
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
236
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
322
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
323
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
324
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
325
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
326
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
327
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
328
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
329
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
330
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
331
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
332
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
333
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
334
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
335
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
336
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
337
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
338
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
339
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
340
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
341
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
342
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
343
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
344
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
429
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
430
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
431
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
432
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
433
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
434
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
435
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
436
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
437
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
438
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
439
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
440
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
441
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
442
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
443
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
444
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
445
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
446
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
447
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
448
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
449
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
450
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
451
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/s